In [2]:
'''
In order for this project to work, matplotlib, numpy and ipympl need to be installed
'''

import numpy as np
from matplotlib import pyplot as plt
%matplotlib widget

In [4]:
'''
Class to represent a Tin

Attributes:
_faces: A list with all the faces (triangles) of the subdivision

Functions, Methods and Attributes taht start with _ are private and should not be called or tampered by the user
'''
class TIN:

    '''
    Subclass to represent a vertex of the Tin

    Attributes:
    _x: the x coordinate of the vertex
    _y: the y coordinate of the vertex
    _height: the height of the vertex
    _inc_edge: an edge with origin this vertex
    '''
    class vertex:
        '''
        Constructor
        x: The x coordinate of the point
        y: the y coordinate of the point
        height: The height of the point, default 0
        inc_edge: an edge with origin this vertex, default None
        '''
        def __init__(self,x,y,height=0,inc_edge=None):
            self._x = x
            self._y = y
            self._height = height
            self._edge = inc_edge

        def __repr__(self):
            return "({},{}), h={}".format(self._x,self._y,self._height)

        def __str__(self):
            return "({},{})".format(self._x,self._y)

        '''
        Defining the sum of two vertices, it ignore the heights
        '''
        def __add__(self,b):
            return TIN.vertex(self._x+b._x,self._y+b._y)

        '''
        Defining the rest of two vertices, it ignore the heights
        '''
        def __sub__(self,b):
            return TIN.vertex(self._x-b._x,self._y-b._y)

        '''
        Defining the equality of two vertices, it ignore the heights
        '''
        def __eq__(self,b):
            return self._x == b._x and self._y == b._y

        '''
        Defining the scalar multiplication of scalar and a vertex, it ignore the height
        '''
        def scalar_mul(self,t):
            return TIN.vertex(self._x*t,self._y*t)

        def dot_prod(self,v):
            return self._x*v._x+self._y*v._y

    '''
    Subclass to represent an edge of a TIN, using the doubly connected edge strategy

    Atributes:
    _origin: A vertex representing the origin of the edge
    _prev: The previous edge on the same face, counterclockwise
    _nxt: The next edge on the same face, counterclockwise
    _inn_face: The inner of the edge in the planar subdivision
    _out_face: The outter face of the edge in the planar subdivision
    _twin: The edge oriented the other way around
    '''
    class edge:
        '''
        Constructor

        Arguments:
        p1: Origin of the edge
        p2: End point of the edge
        twin: If there exists, the edge oriented the other way around
        prev: If there exists, the previous edge
        nxt: If there exists, the next edge
        inn_face: If there exists, the inner face of the edge in the planar subdivision
        out_face: If there exists, the outter face of the edge in the planar subdivision
        '''
        def __init__(self,p1,p2=None,twin=None,prev=None,nxt=None,inn_face=None,out_face=None):
            assert (p2 is None and not twin is None) or (not p2 is None and twin is None), "either p2 or twin has to be None, but not both at the same time"
            self._origin = p1
            self._origin._edge = self
            self._prev = prev
            self._nxt = nxt
            self._face = inn_face
            if twin is None:
                e2 = TIN.edge(p1=p2,p2=None,twin=self)
                self._twin = e2
                self._twin._origin._edge = self._twin
            else:
                self._twin = twin
            if not prev is None:
                self._prev._nxt = self
            if not nxt is None:
                self._nxt._prev = self

        def __repr__(self):
            return "{}->{}".format(self._origin,self._twin._origin)

        def __str__(self):
            return "{}->{}".format(self._origin,self._twin._origin)

        '''
        Define the equivalence between two oriented edges
        '''
        def __eq__(self,e):
            return self._origin == e._origin and self._twin._origin == e._twin._origin

        '''
        Define the dot product
        '''
        def dot_prod(self,b):
            v1 = self._twin._origin-self._origin
            v2 = b._twin._origin-b._origin
            return (v1._x*v2._x)+(v1._y*v2._y)


        '''
        Returns the lenght of the edge, using euclidean distance
        '''
        def size(self):
            v0 = self._origin
            v1 = self._twin._origin
            return ((v0._x-v1._x)**2+(v0._y-v1._y)**2)**0.5

    '''
    Subclass to represent a face (triangle) of the Tin

    Attributes:
    _inc_edge: An edge of the face
    '''
    class face:
        '''
        Constructor

        Arguments:
        inc_edge: An edge of the face
        '''
        def __init__(self,inc_edge):
            self._inc_edge = inc_edge
            self._inc_edge._face = self
            e = self._inc_edge._nxt
            while (e is not None) and not (e == self._inc_edge):
                e._face = self
                e = e._nxt
        
        '''
        Defining equality of faces
        '''
        def __eq__(self,f2):
            edges = []
            e = self._inc_edge
            while e is not None and e not in edges:
                edges.append(e)
                e = e._nxt
            return f2._inc_edge in edges

        
        def __repr__(self):
            edges = []
            e = self._inc_edge
            while e is not None and e not in edges:
                edges.append(e)
                e = e._nxt
            out = "|"
            for edge in edges:
                out += edge.__str__() + '|'
            return out

        def __str__(self):
            edges = []
            e = self._inc_edge
            while e is not None and e not in edges:
                edges.append(e)
                e = e._nxt
            out = "|"
            for edge in edges:
                out += edge.__str__() + '|'
            return out

    '''
    Function that returns four important points, needed for the Delaunay triangulation

    Arguments:
    vertices: list of vertices

    Returns:
    Pymin: vertex with the lowest y coordinate, in case of several points with the same lowest y, the most left
    Pymax: vertex with the greatest y coordinate, in case of several points with the same greatest y, the most left
    Pxleft: point P with minimal angle between the vector(-1,0) and P-(Pymax+(0,1))
    Pright: point P with minimal angle between the vector(1,0) and P-(Pymax+(0,1)), for both Px, only pints with angle less that pi/2 are considered

    Complexity: O(N) in complexity and O(1) in space
    '''
    def _get_minmax_xy(self,vertices):
        Pymin = vertices[0]
        Pymax = vertices[0]
        for p in vertices[1:]:
            if p._y > Pymax._y or (p._y == Pymax._y and p._x < Pymax._x):
                Pymax = p
            if p._y < Pymin._y or (p._y == Pymin._y and p._x < Pymin._x):
                Pymin = p
        cw_minangle = np.pi/2
        ccw_minangle = np.pi/2
        Pxleft = Pymax
        Pxright = Pymax
        for p in vertices:
            vec = p-Pymax-self.vertex(0,1)
            left_angle = np.arccos(-vec._x/(vec._x**2+vec._y**2)**0.5)
            if left_angle <= ccw_minangle:
                ccw_minangle = left_angle
                Pxleft = p
            right_angle = np.arccos(vec._x/(vec._x**2+vec._y**2)**0.5)
            if right_angle < cw_minangle or (right_angle == cw_minangle and p._x>=Pxleft._x):
                cw_minangle = right_angle
                Pxright = p
        return Pymin,Pymax,Pxleft,Pxright

    '''
    Function that builds a triangle containing all vertices at vertices

    Arguments:
    vertices: list of vertices

    Returns:
    None, the triangle is stored at _faces

    Complexity: O(N) in complexity and O(1) in space
    '''
    def _assemble_big_triangle(self,vertices):
        assert (len(vertices)>0), "To construct the big triangle at least 1 vertex is needed"
        Pymin,Pymax,Pxleft,Pxright = self._get_minmax_xy(vertices)
        tL = (Pymin._y-1-Pxleft._y)/(Pxleft._y-Pymax._y-1)
        tR = (Pymin._y-1-Pxright._y)/(Pxright._y-Pymax._y-1)
        v0 = Pymax+self.vertex(0,1)
        v1 = Pxleft+(Pxleft-v0).scalar_mul(tL)-self.vertex(1,0)
        v2 = Pxright+(Pxright-v0).scalar_mul(tR)+self.vertex(1,0)
        e0 = self.edge(v0,v1)
        e1 = self.edge(v1,v2,prev=e0)
        e2 = self.edge(v2,v0,prev=e1,nxt=e0)
        f = self.face(e0)
        self._faces.append(f)


    '''
    Function that determines if a point p is inside the circumference that circumscribes a triangle

    Arguments:
    p: A vertex
    triangle: A face representing the triangle 

    Returns:
    Boolean

    Complexity: O(1)
    '''
    def _inside_circ(self,p,triangle):
        try:
            e = triangle._inc_edge
            v0 = e._origin
            e = e._nxt
            v1 = e._origin
            e = e._nxt
            v2 = e._origin
            assert (v0 == e._nxt._origin), "the face {} isn't a triangle".format(triangle)
            x1,y1 = v0._x,v0._y
            x2,y2 = v1._x,v1._y
            x3,y3 = v2._x,v2._y
            a = (y3-y1)*(x2-x3)-(x1-x3)*(y3-y2)
            assert (not a == 0), "The face {}isn't a valid triangle".format(triangle)
            t = 0.5*((x2-x3)*(x2-x1)+(y2-y3)*(y2-y1))/a
            circ_cent = (v0+v2).scalar_mul(0.5)+self.vertex(t*(y3-y1),t*(x1-x3))
            a = ((v0._x-v1._x)**2+(v0._y-v1._y)**2)**0.5
            b = ((v1._x-v2._x)**2+(v1._y-v2._y)**2)**0.5
            c = ((v2._x-v0._x)**2+(v2._y-v0._y)**2)**0.5
            d = (a+b+c)*(b+c-a)*(c+a-b)*(a+b-c)
            assert(not d == 0), "The face {} isn't a valid triangle".format(triangle)
            circ_rad = a*b*c/d**0.5
            return (p._x-circ_cent._x)**2+(p._y-circ_cent._y)**2 < circ_rad**2
        except:
            assert (1 == 0), "the face {} isn't a triangle".format(triangle)

    '''
    Function that calculates the Delaunay triangulation of a list of vertices

    Arguments:
    vertices: A list of vertices

    Returns:
    None, it stores all the triangles at _faces

    Complexity: O(N^2), O(N) in space
    '''
    def _BowyerWatson(self,vertices):
        assert (len(self._faces) == 1), "There can only be 1 face when calculating the Delaunay triangulation"
        triangles = [self._faces[0]]
        big_triangle = triangles[0]
        big_vertices = [big_triangle._inc_edge._origin,big_triangle._inc_edge._nxt._origin,big_triangle._inc_edge._nxt._nxt._origin]
        for p in vertices:
            bad_triangles = []
            for triangle in triangles:
                try:
                    if (self._inside_circ(p,triangle)):
                        bad_triangles.append(triangle)
                except:
                    bad_triangles.append(triangle)
            for triangle in bad_triangles:
                triangles.remove(triangle)
            polygon = []
            rejected = []
            for triangle in bad_triangles:
                edges = []
                e = triangle._inc_edge
                while e not in edges:
                    edges.append(e)
                    e = e._nxt
                for edge in edges:
                    if edge._twin in polygon:
                        polygon.remove(edge._twin)
                        rejected.append(edge._twin)
                    elif edge in polygon:
                        polygon.remove(edge)
                        rejected.append(edge)
                    if edge._twin not in rejected and edge not in rejected:
                        polygon.append(edge)
            for edge in polygon:
                v0 = edge._origin
                v1 = edge._twin._origin
                v = v1-v0
                vn = p-v1
                cross = np.sign(v._x*vn._y-v._y*vn._x)
                if cross < 0:
                    v0, v1 = v1, v0
                    edge = edge._twin
                e1 = self.edge(v1,p,prev=edge)
                e2 = self.edge(p,v0,prev=e1,nxt=edge)
                f = self.face(inc_edge=edge)
                triangles.append(f)
        self._faces = []
        for triangle in triangles:
            flag = True
            e = triangle._inc_edge
            vertices = [e._origin,e._nxt._origin,e._nxt._origin]
            for v in vertices:
                if v in big_vertices:
                    flag = False
            if flag:
                self._faces.append(triangle)
                        
        

    '''
    Constructor

    Arguments:
    vertices: A list of vertices (numpy.array, tuple, list, etc.), if there are repeated vertices, they'll be removed
    heights: A list with the heights of each vertex, where heights[i] is the height of the vertex vertices[i]

    Complexity: O(N^2)
    '''
    def __init__(self,vertices,heights):
        assert (len(vertices) == len(list(set(vertices)))), "There are repeated vertices"
        assert (len(vertices) == len(heights)), "There are not the same amount of vertices and heights"
        self._faces = []
        ver = []
        for i in range(len(vertices)):
            ver.append(self.vertex(vertices[i][0],vertices[i][1],heights[i]))
        if len(vertices)>0:
            self._assemble_big_triangle(ver)
            self._BowyerWatson(ver)

    '''
    Function that returns all the vertices of the Tin

    Complexity: O(N)
    '''
    def get_vertices(self):
        vertices = []
        for f in self._faces:
            e = f._inc_edge
            while (e is not None) and (not e._nxt == f._inc_edge):
                if e._origin not in vertices:
                    vertices.append(e._origin)
                e = e._nxt
            if (e is not None) and (e._origin not in vertices):
                vertices.append(e._origin)
        return vertices

    '''
    Function to display the Tin using matplotlib.pyplot

    Arguments:
    see_height: if True, height of all vertex is displayed
    '''
    def visualize(self,see_heights=False):
        fig = plt.figure()
        fig.set_size_inches(8,8)
        for f in self._faces:
            e = f._inc_edge
            x = []
            y = []
            while (e is not None) and not (e._nxt == f._inc_edge):
                x.append(e._origin._x)
                y.append(e._origin._y)
                e = e._nxt
            if e is not None:
                x.append(e._origin._x)
                y.append(e._origin._y)
                e = e._nxt
                if e is not None:
                    x.append(e._origin._x)
                    y.append(e._origin._y)
            plt.plot(x,y,'k-')
        vertices = self.get_vertices()
        x = [i._x for i in vertices]
        y = [i._y for i in vertices]
        t = ["{}".format(i._height) for i in vertices]
        plt.plot(x,y,'ko')
        if see_heights:
            for i in range(len(t)):
                plt.text(x[i],y[i],t[i],fontsize=20,color='r')
        plt.show()


    '''
    Function thatd determines if a point pt is inside a triangle

    Arguments:
    pt: A vertex
    triangle: A face representing a triangle

    Returns: True, if pt is inside triangle (including boundaries), False if not

    Complexity: O(1)
    '''
    def _is_inside(self,pt,triangle):
        e = triangle._inc_edge
        v0 = e._origin
        e = e._nxt
        v1 = e._origin
        e = e._nxt
        v2 = e._origin
        e = e._nxt
        assert (v0 == e._origin), "The face isn't a valid triangle"
        d1 = (pt._x - v1._x) * (v0._y - v1._y) - (v0._x - v1._x) * (pt._y - v1._y)
        d2 = (pt._x - v2._x) * (v1._y - v2._y) - (v1._x - v2._x) * (pt._y - v2._y)
        d3 = (pt._x - v0._x) * (v2._y - v0._y) - (v2._x - v0._x) * (pt._y - v0._y)

        has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
        has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)

        return not (has_neg and has_pos)

    '''
    Function that given a point, returns the triangle containing the point, None if the point is outside the TIN

    Arguments:
    pt: a vertex

    Returns: A face (triangle) containing the point, None if is outside the Tin

    Complexity: O(N)
    '''
    def _get_triangle(self,pt):
        for tri in self._faces:
            if self._is_inside(pt,tri):
                return tri
        return None

    '''
    Function that stimates the height of a given point

    Arguments:
    pt: a point

    Returns:
    height of the point

    Complexity: O(N)
    '''
    def get_height(self,pt):
        pt = self.vertex(pt[0],pt[1])
        triangle = self._get_triangle(pt)
        assert (triangle is not None), "The point is outside the polygon"
        v0, v1, v2 = triangle._inc_edge._origin, triangle._inc_edge._nxt._origin, triangle._inc_edge._nxt._nxt._origin
        a = (v2._y-v0._y)*(v1._height-v0._height)-(v1._y-v0._y)*(v2._height-v0._height)
        b = (v1._x-v0._x)*(v2._height-v0._height)-(v2._x-v0._x)*(v1._height-v0._height)
        c = (v2._x-v0._x)*(v1._y-v0._y)-(v1._x-v0._x)*(v2._y-v0._y)
        assert (not c == 0), "The face isn't a valid triangle"
        return (a*(v0._x-pt._x)+b*(v0._y-pt._y)+c*v0._height)/c

    '''
    Function that returns the area of a given triange

    Arguments;
    triangle: A face representing a triangle

    Returns:
    The area of the triangle

    Complexity: O(1)
    '''
    def _area(self,triangle):
        if triangle is None:
            return 0
        v0,v1,v2 = triangle._inc_edge._origin, triangle._inc_edge._nxt._origin, triangle._inc_edge._nxt._nxt._origin
        assert (v0 == triangle._inc_edge._nxt._nxt._nxt._origin), "The face isn't a valid triangle"
        return (v0._x*(v1._y-v2._y)+v1._x*(v2._y-v0._y)+v2._x*(v0._y-v1._y))/2

    '''
    Function that returns the drainage basil of the point pt, which is defined as the biggest polygon generated by concatenate 2 neighbor triangles that contains the point

    Arguments:
    pt: A point

    Returns: 
    A Tin composed by the two triangles

    Complezity: O(N)
    '''
    def largest_drainage_basin(self,pt):
        pt = self.vertex(pt[0],pt[1])
        triangle = self._get_triangle(pt)
        edges = [triangle._inc_edge,triangle._inc_edge._nxt,triangle._inc_edge._nxt._nxt]
        assert (triangle is not None), "The point is outside the Tin"
        neighbor_triangles = []
        for face in self._faces:
            if not face == triangle:
                tmp_edges = [face._inc_edge,face._inc_edge._nxt,face._inc_edge._nxt._nxt]
                for e1 in edges:
                    for e2 in tmp_edges:
                        if e1 == e2._twin:
                            neighbor_triangles.append(face)
        a0 = self._area(triangle)
        areas = [a0+self._area(i) for i in neighbor_triangles]
        i = areas.index(max(areas))
        triangle2 = neighbor_triangles[i]
        points = [i._origin for i in edges]
        heights = [i._height for i in points]
        tmp = [triangle2._inc_edge._origin,triangle2._inc_edge._nxt._origin,triangle2._inc_edge._nxt._nxt._origin]
        for p in tmp:
            if p not in points:
                points.append(p)
                heights.append(p._height)
                break
        points = [(i._x,i._y) for i in points]
        return TIN(points,heights)

    '''
    Function that reports the quality of the triangulation

    Arguments:
    pt: A point

    Returns: 
    The maximum and minimum angle in degrees

    Complezity: O(N)
    '''
    def triangulation_quality(self,pt):
        pt = self.vertex(pt[0],pt[1])
        triangle = self._get_triangle(pt)
        e = triangle._inc_edge
        angles = []
        for i in range(3):
            val = e._twin.dot_prod(e._nxt)
            val = val/(e.size()*e._nxt.size())
            angles.append(np.arccos(val)*180/np.pi)
            e = e._nxt
        return (max(angles),min(angles))

    '''
    Method that plots the TIN in a 3d plot
    '''
    def plot_terrain(self,colormap='Greys'):
        f = plt.figure()
        f.set_size_inches((8,8))
        ax = f.gca(projection="3d")
        vertices = self.get_vertices()
        x = [i._x for i in vertices]
        y = [i._y for i in vertices]
        heights = [i._height for i in vertices]
        ax.plot_trisurf(x,y,heights,cmap=colormap,edgecolor='none',antialiased=True,linewidth=0.2)
        plt.show()

    '''
    Function taht given a vertex, returns all the edges with origin at that vertex

    Arguments:
    pt: a vertex

    Returns:
    A list with all the edges incident to pt

    Complexity: O(N)
    '''
    def _get_edges(self,pt):
        faces = self._faces
        edges = []
        for face in faces:
            e = face._inc_edge
            for i in range(3):
                if e not in edges and e._twin not in edges:
                    if e._origin == pt:
                        edges.append(e)
                    elif e._twin._origin == pt:
                        edges.append(e._twin)
                e = e._nxt
        return edges

    '''
    Function taht given a vertex, returns its flow path

    Arguments:
    pt: a vertex

    Returns:
    A list with all the points that the flow will follow

    Complexity: O(E*N) where E is the complexity of the flowpath
    '''
    def get_flow(self,pt):
        pt = self.vertex(pt[0],pt[1])
        triangle = self._get_triangle(pt)
        if triangle is None:
            return []
        path = [(pt._x,pt._y)]
        visited = []
        v0 = None
        while True:
            visited.append(triangle)
            edges = [triangle._inc_edge,triangle._inc_edge._nxt,triangle._inc_edge._nxt._nxt]
            v = [i._origin for i in edges]
            h = np.array([i._height for i in v])
            indices = np.where(h == min(h))[0]
            if len(indices) == 3:
                return path
            elif len(indices) == 2:
                v0,v1 = v[indices[0]],v[indices[1]]
                N = v1-v0
                N = self.vertex(-N._y/(N._x**2+N._y**2)**0.5,N._x/(N._x**2+N._y**2)**0.5)
                p = pt - N.scalar_mul(N.dot_prod(pt-v0))
                if not p == pt:
                    path.append((p._x,p._y))
                if indices[0] == 0:
                    e1 = edges[indices[1]]
                else:
                    e1 = edges[1]
                flag = False
                for face in self._faces:
                    if not face == triangle:
                        tmp_edges = [face._inc_edge,face._inc_edge._nxt,face._inc_edge._nxt._nxt]
                        for e2 in tmp_edges:
                            if e1 == e2._twin:
                                    triangle = face
                                    flag = True
                    if flag:
                        break
                if triangle in visited:
                    return path
            else:
                v0 = v[indices[0]]
                path.append((v0._x,v0._y))
                break
        while True:
            edges = self._get_edges(v0)
            neighbor_points = [i._twin._origin for i in edges]
            heights = [i._height for i in neighbor_points]
            if v0._height <= min(heights):
                return path
            v0 = neighbor_points[np.argmin(heights)]
            path.append((v0._x,v0._y))
        


In [6]:
vertices = [(6, -5), (3, 3), (-8, -10), (-5, -5), (1, -10), (-9, -7), (-5, 1), (-10, 2), (-1, 10), (5, -2), (0, 10), (-3, -6), (4, 8), (-8, -6), (2, -1), (5, 2), (4, 7), (7, -7), (0, 9), (6, -9)]
heights = [6, 9, 2, 0, 0, 5, 0, 1, 5, 5, 7, 3, 5, 0, 5, 8, 0, 8, 10, 8]
T = TIN(vertices, heights)
T.visualize(see_heights=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
p = (1,1)
print("Height of {} = {}".format(p,T.get_height(p)))
M = T.largest_drainage_basin(p) 
M.visualize(see_heights=True)   

Height of (1, 1) = 5.6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
T.triangulation_quality((4,2))

(77.47119229084849, 30.96375653207352)

In [12]:
T.plot_terrain(colormap='terrain_r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
T.get_flow((-7.8,-1.1))

[(-7.8, -1.1), (-6.194827586206896, -1.7879310344827586)]